<a href="https://colab.research.google.com/github/febriyansyahresearch-lab/Research/blob/main/data-mining/studyplanbot/STUDYPLANBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# StudyPlanBot v5.3 — FIX "jawaban tidak terlihat" (NO JS)
# Chat render ulang dari history -> pasti tampil + bisa di-scroll
# Dark/Light auto | Tanpa API
# ============================================================

import time
from dataclasses import dataclass
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML as DHTML
from datetime import datetime, timedelta

# ---------- Helpers ----------
def clamp(x, lo, hi): return max(lo, min(hi, x))
def esc(s: str) -> str:
    return (s.replace("&","&amp;").replace("<","&lt;").replace(">","&gt;")
              .replace('"',"&quot;").replace("'","&#39;"))

# ---------- Simulasi data ----------
TODAY = datetime.now().date()
CALENDAR = [
    ("Buka KRS", TODAY + timedelta(days=3)),
    ("Tutup KRS", TODAY + timedelta(days=10)),
    ("Batas bayar UKT", TODAY + timedelta(days=7)),
    ("Mulai perkuliahan", TODAY + timedelta(days=14)),
    ("Deadline proposal (simulasi)", TODAY + timedelta(days=30)),
]
SERVICES = {
    "BAAK": "baak@kampus.ac.id",
    "Keuangan": "keuangan@kampus.ac.id",
    "Admin Prodi": "adminprodi@kampus.ac.id",
    "Helpdesk TI": "helpdesk@kampus.ac.id",
    "Perpustakaan": "library@kampus.ac.id",
}
FAQ = {
    "KRS": "📌 KRS: siapkan KHS, persetujuan PA, dan rencana MK. Jika terkunci, cek status registrasi/keuangan.",
    "KHS": "📌 KHS: muncul setelah nilai final diproses. Jika belum ada, cek apakah dosen sudah submit nilai.",
    "UKT": "📌 UKT: jika sudah bayar tapi belum update, simpan bukti bayar & laporkan ke keuangan.",
    "Jadwal": "📌 Jadwal: cek portal akademik / kanal resmi karena bisa berubah (ruang/dosen).",
    "Wisuda": "📌 Wisuda (simulasi): nilai final lengkap, bebas pustaka, bebas administrasi, unggah berkas sesuai jadwal.",
    "Cuti": "📌 Cuti: ajukan sebelum registrasi berakhir. Siapkan alasan, persetujuan PA/kaprodi, dan formulir cuti.",
}

# ---------- Model simulasi ----------
@dataclass
class StudentProfile:
    ipk: float
    hadir: int
    sks: int
    ulang: int
    semester: int
    total_sks: int

def predict_risk(p: StudentProfile):
    ipk_score   = clamp((3.5 - p.ipk) / 2.0, 0.0, 1.0)
    hadir_score = clamp((85 - p.hadir) / 50.0, 0.0, 1.0)
    ulang_score = clamp(p.ulang / 5.0, 0.0, 1.0)
    sks_score   = clamp((18 - p.sks) / 12.0, 0.0, 1.0) if p.sks < 18 else 0.0
    expected    = p.semester * 20
    prog_gap    = clamp((expected - p.total_sks) / 60.0, 0.0, 1.0)

    prob = 0.32*ipk_score + 0.20*hadir_score + 0.18*ulang_score + 0.15*sks_score + 0.15*prog_gap
    prob = float(round(prob, 2))

    if prob >= 0.60:
        return "Tinggi", prob, "✅ Fokus perbaikan IPK, hadir >85%, amankan prasyarat, konsultasi SKS ke PA/pembimbing."
    if prob >= 0.35:
        return "Sedang", prob, "✅ Jaga konsistensi hadir, pilih SKS optimal, kurangi matkul ulang, mulai roadmap skripsi/tesis."
    return "Rendah", prob, "✅ Pertahankan performa dan mulai persiapan topik skripsi/tesis lebih awal."

def recommend_sks(ipk, hadir, ulang):
    if ipk >= 3.5 and hadir >= 90 and ulang == 0:
        return "22–24", "Performa sangat baik. Ambil 22–24 SKS sambil jaga kualitas."
    if ipk >= 3.0 and hadir >= 85 and ulang <= 1:
        return "20–22", "Performa baik. Ambil 20–22 SKS, prioritaskan prasyarat."
    if ipk >= 2.75 and hadir >= 80 and ulang <= 2:
        return "18–20", "Performa cukup. Ambil 18–20 SKS, fokus matkul sulit."
    return "14–16", "Perlu konsolidasi. Ambil 14–16 SKS untuk naikkan IPK & menekan ulang."

def thesis_ready(ipk, semester, total_sks):
    if semester < 3:
        return "Belum", "Masih tahap awal. Stabilkan ritme studi + eksplorasi minat/topik."
    if ipk < 2.75:
        return "Belum", "Naikkan IPK dulu agar proses bimbingan lebih lancar."
    if total_sks < semester * 16:
        return "Belum", "Progres SKS relatif rendah. Amankan prasyarat & susun rencana."
    return "Siap", "Mulai pilih area topik, cari gap riset, dan susun outline proposal."

# ---------- CSS ----------
STYLE = widgets.HTML("""
<style>
:root{
  --bg:#ffffff; --surface:#ffffff; --border:#e8e8e8; --shadow:rgba(0,0,0,.10);
  --text:#111827; --muted:rgba(17,24,39,.70);
  --bot:#f1f0f0; --user:#dcf8c6; --codebg:rgba(0,0,0,.06);
  --hdr1:#0b1320; --hdr2:#162033; --hdrText:#ffffff;
}
@media (prefers-color-scheme: dark){
  :root{
    --bg:#0b0f19; --surface:#111827; --border:rgba(255,255,255,.12); --shadow:rgba(0,0,0,.35);
    --text:rgba(255,255,255,.92); --muted:rgba(255,255,255,.70);
    --bot:rgba(255,255,255,.10); --user:rgba(34,197,94,.22); --codebg:rgba(255,255,255,.10);
    --hdr1:#0a1020; --hdr2:#0f1b33; --hdrText:rgba(255,255,255,.95);
  }
}
.spb{font-family:Arial; color:var(--text); max-width:1100px;}
.hdr{
  background:linear-gradient(90deg,var(--hdr1),var(--hdr2));
  color:var(--hdrText); padding:14px 16px; border-radius:16px;
  box-shadow:0 6px 18px var(--shadow);
  display:flex; justify-content:space-between; align-items:center; gap:12px;
}
.h1{font-size:20px;font-weight:700;}
.h2{font-size:12px; opacity:.92;}
.badge{background:rgba(255,255,255,.14); padding:6px 10px; border-radius:999px; font-size:12px;}

.sectionTitle{font-weight:700; margin:0 0 6px 0;}
.small{color:var(--muted); font-size:12px; margin:0;}

.chatBox{
  background:var(--surface);
  border:1px solid var(--border);
  border-radius:16px;
  padding:10px 12px;
  height:420px;
  overflow:auto;
  box-shadow:0 4px 12px var(--shadow);
}

.row{display:flex; margin:8px 0;}
.row.user{justify-content:flex-end;}
.row.bot{justify-content:flex-start;}
.bub{
  max-width:78%; padding:10px 12px; border-radius:14px; font-size:14px; line-height:1.35;
  box-shadow:0 1px 2px var(--shadow); border:1px solid var(--border); color:var(--text);
}
.bub.user{background:var(--user);}
.bub.bot{background:var(--bot);}
code{background:var(--codebg); padding:2px 6px; border-radius:8px; border:1px solid var(--border);}

.kpiGrid{
  display:grid;
  grid-template-columns: repeat(auto-fit, minmax(160px, 1fr));
  gap:10px;
  margin:10px 0 0 0;
}
.kpiCard{
  background:var(--surface);
  border:1px solid var(--border);
  border-radius:14px;
  padding:10px 12px;
  box-shadow:0 3px 10px var(--shadow);
  min-height:64px;
}
.kpiCard .v{font-size:18px; font-weight:800;}
.kpiCard .l{font-size:12px; color:var(--muted);}
.hr{border-top:1px solid var(--border); margin:10px 0;}
</style>
""")

header = widgets.HTML("""
<div class="spb">
  <div class="hdr">
    <div>
      <div class="h1">🎓 StudyPlanBot</div>
      <div class="h2">Chatbot Konsultasi & Perencanaan Studi Mahasiswa</div>
    </div>
    <div class="badge">Dark/Light Auto • S2 MTI • Data Mining Simulasi</div>
  </div>
</div>
""")

# ---------- Chat: history render (NO JS, pasti tampil) ----------
chat_out = widgets.Output()
history = []  # list of dict: {"who": "bot"/"user", "html": "..."}  (bot boleh html, user di-escape)

def render_chat():
    with chat_out:
        clear_output()
        blocks = []
        for m in history:
            blocks.append(f"<div class='row {m['who']}'><div class='bub {m['who']}'>{m['html']}</div></div>")
        display(DHTML("<div class='chatBox'>" + "".join(blocks) + "</div>"))

def bot(html):
    history.append({"who": "bot", "html": html})
    render_chat()

def user(text):
    history.append({"who": "user", "html": esc(text)})
    render_chat()

def reset_chat():
    history.clear()
    render_chat()

# ---------- KPI ----------
kpi_out = widgets.Output()
def render_kpi(p: StudentProfile, lvl="—", prob="—", sks_range="—"):
    with kpi_out:
        clear_output()
        display(DHTML(f"""
        <div class="kpiGrid">
          <div class="kpiCard"><div class="v">{p.ipk:.2f}</div><div class="l">IPK</div></div>
          <div class="kpiCard"><div class="v">{p.hadir}%</div><div class="l">Kehadiran</div></div>
          <div class="kpiCard"><div class="v">{p.sks}</div><div class="l">SKS Semester Ini</div></div>
          <div class="kpiCard"><div class="v">{lvl} ({prob})</div><div class="l">Risiko</div></div>
          <div class="kpiCard"><div class="v">{sks_range}</div><div class="l">Rekomendasi SKS</div></div>
        </div>
        """))

# ---------- Controls ----------
txt = widgets.Text(placeholder="Ketik: menu | krs | kalender | surat | risiko | sks | skripsi | tiket ...",
                   layout=widgets.Layout(width="100%"))
btn_send = widgets.Button(description="Kirim", button_style="success", icon="paper-plane")
btn_reset = widgets.Button(description="Reset", icon="trash")
btn_menu = widgets.Button(description="Menu", button_style="info", icon="list")
btn_demo = widgets.Button(description="Demo", icon="play")
top_btns = widgets.HBox([btn_menu, btn_demo, btn_reset], layout=widgets.Layout(flex_flow="row wrap"))

qa1 = widgets.Button(description="📊 Risiko", button_style="warning")
qa2 = widgets.Button(description="📚 SKS", button_style="info")
qa3 = widgets.Button(description="📝 Skripsi/Tesis", button_style="info")
qa4 = widgets.Button(description="🗓️ Kalender")
qa5 = widgets.Button(description="✉️ Surat")
qa6 = widgets.Button(description="🎫 Buat Tiket")
quick = widgets.HBox([qa1, qa2, qa3, qa4, qa5, qa6], layout=widgets.Layout(flex_flow="row wrap"))

# Data panel
ipk_w   = widgets.FloatSlider(description="IPK", min=0, max=4, step=0.01, value=3.20, readout_format=".2f")
hadir_w = widgets.IntSlider(description="Hadir %", min=0, max=100, value=85)
sks_w   = widgets.IntSlider(description="SKS", min=0, max=30, value=20)
ulang_w = widgets.IntSlider(description="Ulang", min=0, max=20, value=1)
sem_w   = widgets.IntSlider(description="Semester", min=1, max=20, value=3)
tot_w   = widgets.IntSlider(description="Total SKS", min=0, max=300, value=40)
btn_run = widgets.Button(description="Jalankan", button_style="primary", icon="play")
btn_close = widgets.Button(description="Tutup", icon="times")
form = widgets.VBox([
    widgets.HTML("<div class='sectionTitle'>Panel Data Mahasiswa</div><div class='small'>Isi data untuk analisis Risiko / SKS / Skripsi.</div>"),
    ipk_w, hadir_w, sks_w, ulang_w, sem_w, tot_w,
    widgets.HBox([btn_run, btn_close])
])
form.layout.display = "none"

# Ticket panel
ticket_cat   = widgets.Dropdown(options=["BAAK", "Keuangan", "Admin Prodi", "Helpdesk TI", "Perpustakaan"], description="Kategori")
ticket_title = widgets.Text(placeholder="Judul tiket (contoh: KRS terkunci)", layout=widgets.Layout(width="100%"))
ticket_desc  = widgets.Textarea(placeholder="Deskripsi singkat masalah...", layout=widgets.Layout(width="100%", height="90px"))
btn_ticket_send = widgets.Button(description="Submit Tiket", button_style="warning", icon="check")
btn_ticket_close = widgets.Button(description="Tutup", icon="times")
ticket_box = widgets.VBox([
    widgets.HTML("<div class='sectionTitle'>🎫 Helpdesk Ticket (Simulasi)</div><div class='small'>Tiket disimulasikan (tanpa API) untuk demo.</div>"),
    ticket_cat, ticket_title, ticket_desc,
    widgets.HBox([btn_ticket_send, btn_ticket_close])
])
ticket_box.layout.display = "none"

# ---------- Sidebar ----------
menu_btns = []
def mbtn(label, key, style=""):
    b = widgets.Button(description=label, button_style=style, layout=widgets.Layout(width="100%"))
    b._spb_key = key
    menu_btns.append(b)
    return b

sidebar = widgets.VBox([
    widgets.HTML("<div class='sectionTitle'>📚 Menu Layanan</div><div class='small'>Klik menu untuk menjalankan fitur.</div>"),
    mbtn("Menu Utama", "menu", "info"),
    mbtn("FAQ Akademik (KRS/KHS/UKT)", "faq"),
    mbtn("Kalender Akademik", "kalender"),
    mbtn("Rekomendasi SKS", "sks"),
    mbtn("Prediksi Risiko Keterlambatan", "risiko", "warning"),
    mbtn("Kesiapan Skripsi/Tesis", "skripsi"),
    mbtn("Roadmap Tesis (S2)", "roadmap"),
    mbtn("Template Surat Akademik", "surat"),
    mbtn("Informasi Wisuda", "wisuda"),
    mbtn("Kontak Layanan", "kontak"),
    mbtn("Buat Tiket Helpdesk", "tiket"),
    widgets.HTML("<div class='hr'></div><div class='small'>Tip: ketik juga bisa, contoh <code>kalender</code>, <code>surat</code>, <code>tiket</code>.</div>")
])

# ---------- Actions ----------
state = {"mode": None}

def close_panels():
    form.layout.display = "none"
    ticket_box.layout.display = "none"
    state["mode"] = None

def show_menu():
    bot("<b>Menu Utama</b><br>"
        "• Info: <code>krs</code>, <code>khs</code>, <code>ukt</code>, <code>kalender</code>, <code>surat</code>, <code>wisuda</code>, <code>kontak</code><br>"
        "• Analitik: <code>sks</code>, <code>risiko</code>, <code>skripsi</code>, <code>roadmap</code><br>"
        "• Layanan: <code>tiket</code> (helpdesk simulasi).")

def show_calendar():
    rows = "<br>".join([f"• <b>{name}</b>: {dt.strftime('%d %b %Y')}" for name, dt in CALENDAR])
    bot(f"🗓️ <b>Kalender Akademik (Simulasi)</b><br>{rows}")

def show_contacts():
    rows = "<br>".join([f"• <b>{k}</b>: {v}" for k,v in SERVICES.items()])
    bot(f"☎️ <b>Kontak Layanan</b><br>{rows}")

def show_roadmap():
    bot("🧭 <b>Roadmap Tesis S2 (Ringkas)</b><br>"
        "1) Area & gap riset<br>2) Proposal + RQ<br>3) Metode + instrumen<br>"
        "4) Data collection<br>5) Analisis (Data Mining)<br>6) Penulisan + sidang")

def show_letters():
    bot("✉️ <b>Template Surat</b><br>Ketik: <code>surat aktif</code> / <code>surat penelitian</code> / <code>surat pembimbing</code>")

def open_form(mode):
    close_panels()
    state["mode"] = mode
    form.layout.display = "flex"
    bot("Isi <b>Panel Data Mahasiswa</b> lalu klik <b>Jalankan</b>.")

def open_ticket():
    close_panels()
    ticket_box.layout.display = "flex"
    bot("🎫 Isi form tiket (simulasi) lalu klik <b>Submit Tiket</b>.")

def run_analysis(_=None):
    p = StudentProfile(float(ipk_w.value), int(hadir_w.value), int(sks_w.value),
                       int(ulang_w.value), int(sem_w.value), int(tot_w.value))
    lvl, prob, tips = predict_risk(p)
    sks_range, sks_note = recommend_sks(p.ipk, p.hadir, p.ulang)
    render_kpi(p, lvl, prob, sks_range)

    mode = state["mode"]
    if mode == "risiko":
        bot(f"📊 <b>Risiko:</b> <b>{lvl}</b> (skor <b>{prob}</b>)<br>{tips}")
    elif mode == "sks":
        bot(f"📚 <b>Rekomendasi SKS:</b> <b>{sks_range}</b><br>{sks_note}")
    elif mode == "skripsi":
        st, note = thesis_ready(p.ipk, p.semester, p.total_sks)
        bot(f"📝 <b>Kesiapan Skripsi/Tesis:</b> <b>{st}</b><br>{note}")
    close_panels()

def submit_ticket(_=None):
    cat = ticket_cat.value
    title = ticket_title.value.strip() or "(tanpa judul)"
    desc = ticket_desc.value.strip() or "(tanpa deskripsi)"
    ticket_id = f"TKT-{int(time.time())%100000:05d}"
    bot(f"✅ <b>Tiket dibuat</b><br>ID: <b>{ticket_id}</b><br>Kategori: <b>{cat}</b><br>Judul: {esc(title)}<br>Deskripsi: {esc(desc)}<br>Kontak: {SERVICES.get(cat,'-')}")
    ticket_title.value = ""
    ticket_desc.value = ""
    close_panels()

def handle_text(msg: str):
    t = msg.strip().lower()
    if not t: return
    if t in ["menu","help","bantuan"]: show_menu(); return
    if t in ["krs","khs","ukt"]: bot(FAQ[t.upper()]); return
    if t == "kalender": show_calendar(); return
    if t == "kontak": show_contacts(); return
    if t == "roadmap": show_roadmap(); return
    if t == "faq":
        bot("❓ Ketik: <code>krs</code>, <code>khs</code>, <code>ukt</code>, <code>surat</code>, <code>wisuda</code>, <code>cuti</code>, <code>jadwal</code>"); return
    if t == "surat": show_letters(); return
    if t in ["wisuda","cuti","jadwal"]: bot(FAQ[t.capitalize()]); return
    if t == "risiko": open_form("risiko"); return
    if t == "sks": open_form("sks"); return
    if t in ["skripsi","tesis"]: open_form("skripsi"); return
    if t in ["tiket","helpdesk"]: open_ticket(); return
    bot("Aku belum paham. Ketik <code>menu</code> atau klik menu di sidebar.")

def on_send(_=None):
    msg = txt.value
    txt.value = ""
    if not msg.strip(): return
    user(msg)
    bot("<i>…</i>")
    time.sleep(0.05)
    handle_text(msg)

def on_reset(_=None):
    close_panels()
    reset_chat()
    p = StudentProfile(float(ipk_w.value), int(hadir_w.value), int(sks_w.value),
                       int(ulang_w.value), int(sem_w.value), int(tot_w.value))
    render_kpi(p)
    bot("Chat direset. Ketik <code>menu</code> untuk mulai.")
    show_menu()

def on_demo(_=None):
    user("menu"); show_menu()
    user("kalender"); show_calendar()
    user("risiko"); open_form("risiko")

# Bind
btn_send.on_click(on_send)
txt.on_submit(on_send)
btn_reset.on_click(on_reset)
btn_menu.on_click(lambda _: show_menu())
btn_demo.on_click(on_demo)
btn_run.on_click(run_analysis)
btn_close.on_click(lambda _: close_panels())
btn_ticket_send.on_click(submit_ticket)
btn_ticket_close.on_click(lambda _: close_panels())

qa1.on_click(lambda _: (user("risiko"), open_form("risiko")))
qa2.on_click(lambda _: (user("sks"), open_form("sks")))
qa3.on_click(lambda _: (user("skripsi"), open_form("skripsi")))
qa4.on_click(lambda _: (user("kalender"), show_calendar()))
qa5.on_click(lambda _: (user("surat"), show_letters()))
qa6.on_click(lambda _: (user("tiket"), open_ticket()))

def sidebar_click(btn):
    key = getattr(btn, "_spb_key", "menu")
    user(btn.description)
    if key == "menu": show_menu()
    elif key == "faq": bot("❓ Ketik: <code>krs</code>, <code>khs</code>, <code>ukt</code>, <code>surat</code>, <code>wisuda</code>, <code>cuti</code>, <code>jadwal</code>")
    elif key == "kalender": show_calendar()
    elif key == "sks": open_form("sks")
    elif key == "risiko": open_form("risiko")
    elif key == "skripsi": open_form("skripsi")
    elif key == "roadmap": show_roadmap()
    elif key == "surat": show_letters()
    elif key == "wisuda": bot(FAQ["Wisuda"])
    elif key == "kontak": show_contacts()
    elif key == "tiket": open_ticket()
    else: show_menu()

for b in menu_btns:
    b.on_click(lambda btn: sidebar_click(btn))

# ---------- Layout ----------
sidebar.layout = widgets.Layout(border="1px solid var(--border)", padding="12px", border_radius="16px", width="340px")
right_box = widgets.VBox([
    widgets.HTML("<div class='sectionTitle'>💬 Chat</div><div class='small'>Sidebar kiri = menu. Hasil analisis tampil di kartu ringkasan.</div>"),
    top_btns,
    quick,
    kpi_out,
    chat_out,
    form,
    ticket_box,
    widgets.HBox([txt, btn_send], layout=widgets.Layout(width="100%"))
])
right_box.layout = widgets.Layout(border="1px solid var(--border)", padding="12px", border_radius="16px", width="100%")

# init
reset_chat()
p0 = StudentProfile(float(ipk_w.value), int(hadir_w.value), int(sks_w.value),
                    int(ulang_w.value), int(sem_w.value), int(tot_w.value))
render_kpi(p0)
bot("Halo! Aku <b>StudyPlanBot</b> 👋<br>Ketuk menu di <b>sidebar</b> atau ketik <code>menu</code>.")
show_menu()

display(widgets.VBox([STYLE, header, widgets.HBox([sidebar, right_box], layout=widgets.Layout(width="100%", gap="12px"))]))
